# Use OpenMMon the HPC

# Non-interactive usage (job script)

**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**

- Copy this notebook and the file [../02/alanine-dipeptide.pdb](../02/alanine-dipeptide.pdb) to a directory on the HPC.
- Create a job script `job_openmm.sh` to run the notebook, with the following contents. In this job script, you must change `mynotebook.ipynb` to the name of your notebook.
You can also modify the number of CPU cores by changing `ppn=9` and the maximum wall time the job is allowed to run. (It will be killed in case your calculation does not stop in time.)


```bash
#!/usr/bin/env bash
#PBS -N _openmm
#PBS -l nodes=1:ppn=9
#PBS -l walltime=5:00:00

# Activate OpenMM.
source ${VSC_DATA}/miniconda/bin/activate
conda activate openmm

# Go to the directory where qsub was executed.
cd ${PBS_O_WORKDIR}

# Set the number of threads.
export OPENMM_CPU_THREADS=${SLURM_CPUS_ON_NODE}

# Run the notebook. (Put everyting on a single line.)
time jupyter nbconvert --to notebook --execute --allow-errors --ExecutePreprocessor.timeout=-1 mynotebook.ipynb
```

- After creating the job script, run `qsub job_openmm.sh`.

In [ ]:
# Import all the modules we need.
from sys import stdout

from openmm import *
from openmm.app import *
from openmm.unit import *

The following code was taken from [../02/02_alanine_dipeptide.ipynb](../02/02_alanine_dipeptide.ipynb), example 3.

In [ ]:
pdb = PDBFile('alanine-dipeptide.pdb')
modeller = Modeller(pdb.topology, pdb.positions)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer)
print(modeller.topology)
# Write a PDB file to provide a topology of the solvated
# system to MDTraj below.
with open('init3.pdb', 'w') as outfile:
    PDBFile.writeFile(modeller.topology, modeller.positions, outfile)

# The modeller builds a periodic box with the solute and solvent molecules.
# PME is the method to compute long-range electristatic interactions in
# periodic systems.
system = forcefield.createSystem(
    modeller.topology, nonbondedMethod=PME, constraints=HBonds)
temperature = 300 * kelvin
pressure = 1 * bar
integrator = LangevinIntegrator(temperature, 1/picosecond, 2*femtoseconds)
system.addForce(MonteCarloBarostat(pressure, temperature))
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('traj3.dcd', 100))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
                                              temperature=True, elapsedTime=True))
simulation.reporters.append(StateDataReporter("scalars3.csv", 100, time=True,
                                              potentialEnergy=True, totalEnergy=True, temperature=True))
simulation.step(100000)

## Interactive usage with on-demand notebooks

TODO